# The Image Registration Problem

Image registration is a core topic of computational anatomy that is applicable across a wide range of contexts. For example, image registration is necessary when:

- Aligning fMRI volumes to correct for motion.
- Aligning longitudinal datasets together to assess change over time.
- Aligning data from different scanners and different sites in multi-centre research.
- Aligning data with a template or atlas to localise results and make comparisons across studies.

For all these cases, we can break the problem down into registering *two* images. This is our general framework for registration, where the goal is to move one image to align with the other. The image we move is known as the *source* image and the image we keep stationary is known as the *target* or *reference* image. 

## Motivation for Registration
To motivate the need for registration, consider the scans shown below.

<iframe src="misaligned.html" width="800" height="600" frameborder="0" scrolling="no" title=""></iframe>

Clearly these images are not aligned because the anatomy of the brain does not overlap. In this situation, our aim is to collate information across the two images. In order to do this, there are two necessary conditions. 

The first is that some sort of *mapping* exists between the points in one image and the points in the other. If the images are identical in size, this is simply an identity mapping where we use the same coordinates for the two images. This will be the case if the images are generated from the same scanning sequence, or if one image has been *resampled* to the dimensions of the other image (as discussed last week). Alternatively, we can map between the images using world-space and the matrix $\mathbf{Q}$, as also explored last week. 

However, mapping between two images is useless if the anatomy of the brain is not in the same position within each image. As such, the second condition is that the images are *aligned* or *registered*. To see this in context, consider reading two images into `MATLAB`